In [10]:
import pandas as pd

dataset_1=pd.read_csv("data/enron_spam_data.csv")
dataset_1.tail()

,Message ID,Subject,Message,Spam/Ham,Date
33711,33711,= ? iso - 8859 - 1 ? q ? good _ news _ c = eda...,"hello , welcome to gigapharm onlinne shop .\np...",spam,2005-07-29
33712,33712,all prescript medicines are on special . to be...,i got it earlier than expected and it was wrap...,spam,2005-07-29
33713,33713,the next generation online pharmacy .,are you ready to rock on ? let the man in you ...,spam,2005-07-30
33714,33714,bloow in 5 - 10 times the time,learn how to last 5 - 10 times longer in\nbed ...,spam,2005-07-30
33715,33715,"dear sir , i am interested in it",hi : )\ndo you need some softwares ? i can giv...,spam,2005-07-31


In [11]:
dataset_1=dataset_1[["Message ID", "Subject", "Message", "Spam/Ham"]]
dataset_1.rename(columns={"Spam/Ham":"Label"}, inplace=True)
dataset_1['Label']=dataset_1['Label'].map({'ham':0, 'spam':1})
dataset_1.dropna(inplace=True)
dataset_1.reset_index(drop=True, inplace=True)
dataset_1["Body"]=dataset_1["Subject"]+" "+dataset_1["Message"]
dataset_1=dataset_1[["Message ID", "Body", "Label"]]
dataset_1.head()

,Message ID,Body,Label
0,1,"vastar resources , inc . gary , production fro...",0
1,2,calpine daily gas nomination - calpine daily g...,0
2,3,re : issue fyi - see note below - already done...,0
3,4,meter 7268 nov allocation fyi .\n- - - - - - -...,0
4,5,"mcmullen gas for 11 / 99 jackie ,\nsince the i...",0


In [14]:
dataset_2=pd.read_csv("data/completeSpamAssassin.csv")
dataset_2.rename(columns={"Unnamed: 0":"Message ID", "text":"Body", "class":"Label"}, inplace=True)
dataset_2.head()

,Message ID,Body,Label
0,0,\nSave up to 70% on Life Insurance.\nWhy Spend...,1
1,1,1) Fight The Risk of Cancer!\nhttp://www.adcli...,1
2,2,1) Fight The Risk of Cancer!\nhttp://www.adcli...,1
3,3,##############################################...,1
4,4,I thought you might like these:\n1) Slim Down ...,1


In [15]:
dataset3=pd.read_csv("data/lingSpam.csv")
dataset3.rename(columns={"Unnamed: 0":"Message ID", "text":"Body", "class":"Label"}, inplace=True)
dataset3.head()

,Message ID,Body,Label
0,0,Subject: great part-time or summer job !\n \n ...,1
1,1,Subject: auto insurance rates too high ?\n \n ...,1
2,2,Subject: do want the best and economical hunti...,1
3,3,Subject: email 57 million people for $ 99\n \n...,1
4,4,Subject: do n't miss these !\n \n attention ! ...,1


In [16]:
full_dataset=pd.concat([dataset_1, dataset_2, dataset3], ignore_index=True)
full_dataset.tail()

,Message ID,Body,Label
41753,2600,Subject: computationally - intensive methods i...,0
41754,2601,Subject: books : a survey of american linguist...,0
41755,2602,Subject: wecol ' 98 - - western conference on ...,0
41756,2603,Subject: euralex ' 98 - revised programme\n \n...,0
41757,2604,",Body,Label\n 0,""Subject: great part-time or s...",0


In [17]:
full_dataset["Label"].value_counts()

Label
0    22815
1    18943
Name: count, dtype: int64

In [ ]:
full_dataset.dropna(inplace=True)

In [33]:
full_dataset.isna().sum()

Body         1
Label        0
label_enc    0
dtype: int64

In [35]:
full_dataset.reset_index(drop=True, inplace=True)
full_dataset.isnull().sum()

Body         1
Label        0
label_enc    0
dtype: int64

In [ ]:
full_dataset.dropna(inplace=True)
full_dataset.reset_index(drop=True, inplace=True)
full_dataset.drop_duplicates(subset=["Body"], inplace=True)
full_dataset.drop(columns=["Message ID"], inplace=True)
full_dataset.head()

,Body,Label
0,"vastar resources , inc . gary , production fro...",0
1,calpine daily gas nomination - calpine daily g...,0
2,re : issue fyi - see note below - already done...,0
3,meter 7268 nov allocation fyi .\n- - - - - - -...,0
4,"mcmullen gas for 11 / 99 jackie ,\nsince the i...",0


In [37]:
# Check for any remaining NA values after all cleaning
full_dataset.isna().sum()

Body         1
Label        0
label_enc    0
dtype: int64

In [32]:
full_dataset.shape

(37920, 3)

In [19]:
import re
import unicodedata

def clean_text(text):
    text = unicodedata.normalize("NFKD", text)
    text = text.lower()
    text = re.sub(r"http\S+|www\S+", "", text)
    text = re.sub(r"<.*?>", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

In [20]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
full_dataset["label_enc"] = label_encoder.fit_transform(full_dataset["Label"])

In [21]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(full_dataset, test_size=0.2, stratify=full_dataset["label_enc"], random_state=42)
print(f"Train shape: {train_df.shape}, Test shape: {test_df.shape}")

Train shape: (30336, 3), Test shape: (7584, 3)


In [22]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def tokenize(texts):
    return tokenizer(
        texts,
        padding=True,
        truncation=True,
        max_length=512,
        return_tensors="pt"
    )

In [ ]:
from torch.utils.data import Dataset, DataLoader
import torch

class SpamEmailDataset(Dataset):
    def __init__(self, df):
        self.texts = df["Body"].tolist()
        self.labels = df["label_enc"].tolist()

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        # Return raw data - tokenization will happen in collate_fn
        return {
            "text": self.texts[idx],
            "label": self.labels[idx]
        }

def collate_fn(batch):
    """Tokenize all texts in batch together for uniform padding"""
    texts = [str(item["text"]) if item["text"] is not None else "" for item in batch]
    labels = [item["label"] for item in batch]
    
    encoded = tokenizer(
        texts,
        padding=True,
        truncation=True,
        max_length=512,
        return_tensors="pt"
    )

    return {
        "input_ids": encoded["input_ids"],
        "labels": torch.tensor(labels, dtype=torch.long)
    }


In [24]:
train_dataloader = DataLoader(SpamEmailDataset(train_df), batch_size=8, shuffle=True, collate_fn=collate_fn)
test_dataloader = DataLoader(SpamEmailDataset(test_df), batch_size=8, shuffle=False, collate_fn=collate_fn)

In [25]:
next(iter(train_dataloader))

{'input_ids': tensor([[  101,  9915, 21711,  ...,     0,     0,     0],
         [  101,  6522, 10531,  ...,  2013,  1024,   102],
         [  101,  1022,  1060,  ...,     0,     0,     0],
         ...,
         [  101,  3395,  1024,  ...,     0,     0,     0],
         [  101,  2006, 21981,  ...,     0,     0,     0],
         [  101, 13921,  1001,  ...,     0,     0,     0]]),
 'labels': tensor([1, 1, 1, 0, 0, 1, 0, 0])}

In [26]:
import torch.nn as nn


class SpamEmailModel(nn.Module):
    def __init__(self, num_classes, vocab_size=30522, embed_dim=256, hidden_dim=256, num_layers=1, rnn_type="gru", bidirectional=True):
        super().__init__()
        self.bidirectional = bidirectional
        dir_mult = 2 if bidirectional else 1

        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=tokenizer.pad_token_id)
        if rnn_type == "lstm":
            self.rnn = nn.LSTM(embed_dim, hidden_dim, num_layers=num_layers, batch_first=True, bidirectional=bidirectional, dropout=0.5)
        elif rnn_type == "gru":
            self.rnn = nn.GRU(embed_dim, hidden_dim, num_layers=num_layers, batch_first=True, bidirectional=bidirectional, dropout=0.5)  # add dropout=0.5
        else:
            self.rnn = nn.RNN(embed_dim, hidden_dim, num_layers=num_layers, batch_first=True, bidirectional=bidirectional)
        self.classifier = nn.Linear(hidden_dim * dir_mult, num_classes)

    def forward(self, input_ids):
        # Compute lengths from non-pad tokens so we don't rely on an external attention_mask
        lengths = (input_ids != tokenizer.pad_token_id).sum(dim=1)

        # Embed and pack for variable-length handling
        embedded = self.embedding(input_ids)
        packed = nn.utils.rnn.pack_padded_sequence(embedded, lengths.cpu(), batch_first=True, enforce_sorted=False)
        packed_out, hidden = self.rnn(packed)

        # Grab last hidden state(s)
        if isinstance(hidden, tuple):  # LSTM returns (h, c)
            hidden = hidden[0]

        if self.bidirectional:
            hidden_fwd = hidden[-2]
            hidden_bwd = hidden[-1]
            h_cat = torch.cat([hidden_fwd, hidden_bwd], dim=1)
        else:
            h_cat = hidden[-1]

        logits = self.classifier(h_cat)
        return logits   

In [27]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

num_classes = len(label_encoder.classes_)
model = SpamEmailModel(
    num_classes=num_classes,
    vocab_size=tokenizer.vocab_size,
    embed_dim=256,
    hidden_dim=256,
    num_layers=1,
    rnn_type="gru",  # options: "gru", "lstm", "rnn"
    bidirectional=True  # set to False for a unidirectional RNN
).to(device)


c:\Users\LENOVO\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\nn\modules\rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn(


In [28]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-3, weight_decay=1e-4)
EPOCHS = 5

In [ ]:
from tqdm import tqdm

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    
    for batch in tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{EPOCHS}"):
        optimizer.zero_grad()
        
        input_ids = batch["input_ids"].to(device)
        labels = batch["labels"].to(device)
        
        outputs = model(input_ids)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_dataloader)

    # Evaluation phase
    model.eval()
    eval_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in test_dataloader:
            input_ids = batch["input_ids"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(input_ids)
            loss = criterion(outputs, labels)
            eval_loss += loss.item()

            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    avg_eval_loss = eval_loss / len(test_dataloader)
    eval_acc = correct / total if total > 0 else 0.0

    print(f"Epoch {epoch+1} | Train Loss: {avg_loss:.4f} | Val Loss: {avg_eval_loss:.4f} | Val Acc: {eval_acc:.4f}")


In [38]:
def predict(texts):
    model.eval()
    encoded = tokenizer(
        texts,
        padding=True,
        truncation=True,
        max_length=512,
        return_tensors="pt"
    )
    input_ids = encoded["input_ids"].to(device)

    with torch.no_grad():
        outputs = model(input_ids)
        probs = nn.Softmax(dim=1)(outputs)
        preds = probs.argmax(dim=1).cpu().numpy()

    return label_encoder.inverse_transform(preds)

In [ ]:
print(predict('''Subject:  CONGRATULATIONS! You’ve WON $1,000,000 USD 

Dear Lucky Winner,
We are pleased to inform you that your email address has been randomly selected as the GRAND PRIZE WINNER in our International Email Lottery Program 2026.
 Prize Amount: $1,000,000 USD
 Payment Method: Bank Transfer / Crypto / Mobile Banking
 Claim Deadline: 48 HOURS ONLY!

To claim your winnings, please reply immediately with the following details:

Full Name

Date of Birth

Country of Residence

Phone Number

Bank Account Details

 IMPORTANT: Failure to respond within 48 hours will result in forfeiture of your prize.

This is a confidential message. Do NOT share this information with anyone.

Congratulations once again on your life-changing opportunity!

Best Regards,
Mr. James Anderson
Senior Claims Officer
Global Prize Commission
claimoffice@global-prize-claim.net'''
))